In [1]:
# Copyright 2019-2024 Quantinuum
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
##
#     http://www.apache.org/licenses/LICENSE-2.0
##
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Conversion from to tket circuits to QuEST circuits
"""
import numpy as np

import pyquest.unitaries as gates
from pyquest.gates import M as Measurement
from pyquest import Circuit as PyQuESTCircuit

from pytket.circuit import Circuit, OpType
from pytket.passes import FlattenRegisters


_ONE_QUBIT_GATES = {
    OpType.X: gates.NOT,
    OpType.Y: gates.Y,
    OpType.Z: gates.Z,
    OpType.H: gates.H,
    OpType.S: gates.S,
    OpType.T: gates.T,
}

_ONE_QUBIT_ROTATIONS = {OpType.Rx: gates.Rx, OpType.Ry: gates.Ry, OpType.Rz: gates.Rz}

_MEASURE_GATES = {OpType.Measure: Measurement}

_TWO_QUBIT_GATES = {OpType.CX: gates.X, OpType.CZ: gates.Z, OpType.SWAP: gates.Swap}


def tk_to_quest(
    circuit: Circuit, reverse_index: bool = False, replace_implicit_swaps: bool = False
) -> PyQuESTCircuit:
    """Convert a pytket circuit to a quest circuit object."""
    circ = circuit.copy()

    if not circ.is_simple:
        FlattenRegisters().apply(circ)

    if replace_implicit_swaps:
        circ.replace_implicit_wire_swaps()
    n_qubits = circ.n_qubits
    quest_operators = []
    index_map = {
        i: (i if not reverse_index else n_qubits - 1 - i) for i in range(n_qubits)
    }
    for com in circ:
        optype = com.op.type
        if optype in _ONE_QUBIT_GATES:
            quest_gate = _ONE_QUBIT_GATES[optype]
            index = index_map[com.qubits[0].index[0]]
            add_gate = quest_gate(index)
            quest_operators.append(add_gate)

        elif optype in _ONE_QUBIT_ROTATIONS:
            quest_gate = _ONE_QUBIT_ROTATIONS[optype]
            index = index_map[com.qubits[0].index[0]]
            param = com.op.params[0] * np.pi
            add_gate = quest_gate(index, param)
            quest_operators.append(add_gate)

        elif optype in _TWO_QUBIT_GATES:
            id1 = index_map[com.qubits[0].index[0]]
            id2 = index_map[com.qubits[1].index[0]]
            quest_gate = _TWO_QUBIT_GATES[optype]
            if optype == OpType.SWAP:
                add_gate = quest_gate(targets=[id1, id2])
            elif optype == OpType.CZ:
                add_gate = gates.Z(id2, controls=id1)
            else:
                add_gate = quest_gate(id2, controls=id1)
            quest_operators.append(add_gate)

        elif optype in _MEASURE_GATES:
            continue

        elif optype == OpType.Barrier:
            continue

        else:
            raise NotImplementedError(
                "Gate: {} Not Implemented in QuEST!".format(optype)
            )

    quest_circ = PyQuESTCircuit(quest_operators)
    return quest_circ



In [2]:

# Copyright 2019-2024 Quantinuum
#s
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

"""Methods to allow tket circuits to be ran on the QuEST simulator
"""

from typing import List, Sequence, Optional, Type, Union,  cast
from logging import warning
from uuid import uuid4
from random import Random
from pytket.backends import (
    Backend,
    CircuitNotRunError,
    CircuitStatus,
    ResultHandle,
    StatusEnum,
)
from pytket.backends.resulthandle import _ResultIdTuple
from pytket.backends.backendinfo import BackendInfo
from pytket.backends.backendresult import BackendResult
from pytket.circuit import Circuit, OpType
from pytket.extensions.qulacs._metadata import __extension_version__
from pytket.passes import (
    BasePass,
    SynthesiseTket,
    SequencePass,
    DecomposeBoxes,
    FullPeepholeOptimise,
    FlattenRegisters,
    auto_rebase_pass,
)
from pytket.predicates import (
    GateSetPredicate,
    NoClassicalControlPredicate,
    NoFastFeedforwardPredicate,
    NoMidMeasurePredicate,
    NoSymbolsPredicate,
    DefaultRegisterPredicate,
    Predicate,
)

from quest_convert import (
    tk_to_quest,
    #_IBM_GATES,
    _MEASURE_GATES,
    _ONE_QUBIT_GATES,
    _TWO_QUBIT_GATES,
    _ONE_QUBIT_ROTATIONS,
)

import pyquest
from pyquest import  QuESTEnvironment, Register
from pyquest.unitaries import *
import numpy as np

_1Q_GATES = (
    set(_ONE_QUBIT_ROTATIONS)
    | set(_ONE_QUBIT_GATES)
    | set(_MEASURE_GATES)
    #| set(_IBM_GATES)
)

class QuESTBackend(Backend):
    """
    Backend for running simulations on the QuEST simulator
    """

    _supports_shots = False
    _supports_counts = False
    _supports_state = True
    _supports_unitary = False
    _supports_density_matrix = True
    _supports_expectation = False
    _expectation_allows_nonhermitian = False
    _supports_contextual_optimisation = False
    _persistent_handles = False
    _GATE_SET = {
        *_TWO_QUBIT_GATES.keys(),
        *_1Q_GATES,
        OpType.Barrier,
    }

    def __init__(
        self,
        result_type: str = "state_vector",
    ) -> None:
        """
        Backend for running simulations on the QuEST simulator

        :param result_type: Indicating the type of the simulation result
            to be returned. It can be either "state_vector" or "density_matrix".
            Defaults to "state_vector"
        """
        super().__init__()
        self._backend_info = BackendInfo(
            type(self).__name__,
            None,
            __extension_version__,
            None,
            self._GATE_SET,
        )
        self._result_type = result_type
        self._sim: Type[Union[Register]]
        self._sim = Register
        if result_type == "state_vector":
            self._density_matrix = False
            self._supports_density_matrix = False
        elif result_type == "density_matrix":
            self._density_matrix = True
            self._supports_state = False
            self._supports_density_matrix = True
        else:
            raise ValueError(f"Unsupported result type {result_type}")

    @property
    def _result_id_type(self) -> _ResultIdTuple:
        return (str,)

    @property
    def backend_info(self) -> Optional["BackendInfo"]:
        return self._backend_info

    @property
    def required_predicates(self) -> List[Predicate]:
        return [
            NoClassicalControlPredicate(),
            NoFastFeedforwardPredicate(),
            NoMidMeasurePredicate(),
            NoSymbolsPredicate(),
            GateSetPredicate(self._GATE_SET),
            DefaultRegisterPredicate(),
        ]

    def rebase_pass(self) -> BasePass:
        return auto_rebase_pass(set(_TWO_QUBIT_GATES) | _1Q_GATES)

    def default_compilation_pass(self, optimisation_level: int = 1) -> BasePass:
        assert optimisation_level in range(3)
        if optimisation_level == 0:
            return SequencePass(
                [DecomposeBoxes(), FlattenRegisters(), self.rebase_pass()]
            )
        elif optimisation_level == 1:
            return SequencePass(
                [
                    DecomposeBoxes(),
                    FlattenRegisters(),
                    SynthesiseTket(),
                    self.rebase_pass(),
                ]
            )
        else:
            return SequencePass(
                [
                    DecomposeBoxes(),
                    FlattenRegisters(),
                    FullPeepholeOptimise(),
                    self.rebase_pass(),
                ]
            )

    def process_circuits(
        self,
        circuits: Sequence[Circuit],
        n_shots: int | Sequence[int] | None = None,
        valid_check: bool = True,
        **kwargs: int | float | str | None,
    ) -> List[ResultHandle]:
        circuits = list(circuits)

        if valid_check:
            self._check_all_circuits(circuits, nomeasure_warn=False)

        handle_list = []
        for circuit in circuits:
            quest_state = self._sim(circuit.n_qubits, self._density_matrix)
            quest_circ = tk_to_quest(
                circuit, reverse_index=True, replace_implicit_swaps=True
            )
            quest_state.apply_circuit(quest_circ)

            if self._result_type == "state_vector":
                state = quest_state[:]  # type: ignore
            else:
                state = quest_state[:, :]  # type: ignore
            qubits = sorted(circuit.qubits, reverse=False)

            if self._result_type == "state_vector":
                try:
                    phase = float(circuit.phase)
                    coeff = np.exp(phase * np.pi * 1j)
                    state *= coeff
                except TypeError:
                    warning(
                        "Global phase is dependent on a symbolic parameter, so cannot "
                        "adjust for phase"
                    )
            handle = ResultHandle(str(uuid4()))
            if self._result_type == "state_vector":
                self._cache[handle] = {
                    "result": BackendResult(state=state, q_bits=qubits)
                }
            else:
                self._cache[handle] = {
                    "result": BackendResult(density_matrix=state, q_bits=qubits)
                }

            handle_list.append(handle)
            del quest_state
            del quest_circ
        return handle_list

    def circuit_status(self, handle: ResultHandle) -> CircuitStatus:
        if handle in self._cache:
            return CircuitStatus(StatusEnum.COMPLETED)
        raise CircuitNotRunError(handle)

In [3]:
# Copyright 2020-2024 Quantinuum
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

from collections import Counter
from typing import List, Sequence, Union, Optional, Dict, Any
import warnings
import math
from datetime import timedelta
from hypothesis import given, strategies, settings
import numpy as np
import pytest
from pytket.backends import ResultHandle
from pytket.circuit import Circuit, BasisOrder, OpType, Qubit
from pytket.pauli import Pauli, QubitPauliString
from pytket.passes import CliffordSimp
from pytket.utils.operators import QubitPauliOperator
from pytket.utils.results import KwargTypes

PARAM = -0.11176849
backends = [
    QuESTBackend(result_type="state_vector"),
    QuESTBackend(result_type="density_matrix"),
]

#OK
def h2_1q_circ(theta: float) -> Circuit:
    circ = Circuit(1)
    circ.Ry(-2 / np.pi * -theta, 0)
    return circ

#OK
def h2_2q_circ(theta: float) -> Circuit:
    circ = Circuit(2).X(0)
    circ.Rx(0.5, 0).H(1)
    circ.CX(0, 1)
    circ.Rz((-2 / np.pi) * theta, 1)
    circ.CX(0, 1)
    circ.Rx(-0.5, 0).H(1)
    return circ

#OK
def h2_3q_circ(theta: float) -> Circuit:
    circ = Circuit(3).X(0).X(1)
    circ.Rx(0.5, 0).H(1).H(2)
    circ.CX(0, 1).CX(1, 2)
    circ.Rz((-2 / np.pi) * theta, 2)
    circ.CX(1, 2).CX(0, 1)
    circ.Rx(-0.5, 0).H(1).H(2)
    return circ

#OK
def h2_4q_circ(theta: float) -> Circuit:
    circ = Circuit(4).X(0).X(1)
    circ.Rx(0.5, 0).H(1).H(2).H(3)
    circ.CX(0, 1).CX(1, 2).CX(2, 3)
    circ.Rz((-2 / np.pi) * theta, 3)
    circ.CX(2, 3).CX(1, 2).CX(0, 1)
    circ.Rx(-0.5, 0).H(1).H(2).H(3)
    return circ

#OK
def test_properties() -> None:
    svb = QuESTBackend()
    dmb = QuESTBackend(result_type="density_matrix")
    assert not svb._density_matrix
    assert dmb._density_matrix

#OK
def test_get_state() -> None:
    quest_circ = h2_4q_circ(PARAM)
    correct_state = np.array(
        [
            -4.97881051e-19 + 3.95546482e-17j,
            -2.04691245e-17 + 4.26119488e-18j,
            -2.05107665e-17 - 1.16628720e-17j,
            -1.11535930e-01 - 2.20309881e-16j,
            1.14532773e-16 + 1.84639112e-16j,
            -2.35945152e-18 + 1.00839027e-17j,
            -3.27177146e-18 - 1.35977120e-17j,
            1.68171141e-17 - 3.67997979e-17j,
            6.96542384e-18 + 6.20603820e-17j,
            2.94777720e-17 + 1.82756571e-19j,
            1.43716480e-17 + 3.62382653e-18j,
            3.41937038e-17 - 8.77511869e-18j,
            9.93760402e-01 + 1.59594560e-15j,
            -2.73151084e-18 + 6.31487294e-17j,
            2.09501038e-17 + 6.22364095e-17j,
            -8.59510231e-18 + 5.90202794e-18j,
        ]
    )
    for b in backends:
        quest_circ = b.get_compiled_circuit(quest_circ)
        if b.supports_state:
            quest_state = b.run_circuit(quest_circ).get_state()
            assert np.allclose(quest_state, correct_state)
        if b.supports_density_matrix:
            quest_state = b.run_circuit(quest_circ).get_density_matrix()
            assert np.allclose(
                quest_state, np.outer(correct_state, correct_state.conj())
            )

#OK
def test_statevector_phase() -> None:
    for b in backends:
        if not b.supports_state:
            continue
        circ = Circuit(2)
        circ.H(0).CX(0, 1)
        circ = b.get_compiled_circuit(circ)
        state = b.run_circuit(circ).get_state()
        assert np.allclose(state, [math.sqrt(0.5), 0, 0, math.sqrt(0.5)], atol=1e-10)
        circ.add_phase(0.5)
        state1 = b.run_circuit(circ).get_state()
        assert np.allclose(state1, state * 1j, atol=1e-10)
        
#OK
def test_swaps_basisorder() -> None:
    # Check that implicit swaps can be corrected irrespective of BasisOrder
    for b in backends:
        c = Circuit(4)
        c.X(0)
        c.CX(0, 1)
        c.CX(1, 0)
        CliffordSimp(True).apply(c)
        assert c.n_gates_of_type(OpType.CX) == 1
        c = b.get_compiled_circuit(c)
        res = b.run_circuit(c)
        if b.supports_state:
            s_ilo = res.get_state(basis=BasisOrder.ilo)
            s_dlo = res.get_state(basis=BasisOrder.dlo)
            correct_ilo = np.zeros((16,))
            correct_ilo[4] = 1.0
            assert np.allclose(s_ilo, correct_ilo)
            correct_dlo = np.zeros((16,))
            correct_dlo[2] = 1.0
            assert np.allclose(s_dlo, correct_dlo)
        if b.supports_density_matrix:
            s_ilo = res.get_density_matrix(basis=BasisOrder.ilo)
            s_dlo = res.get_density_matrix(basis=BasisOrder.dlo)
            correct_ilo = np.zeros((16,))
            correct_ilo[4] = 1.0
            assert np.allclose(s_ilo, np.outer(correct_ilo, correct_ilo.conj()))
            correct_dlo = np.zeros((16,))
            correct_dlo[2] = 1.0
            assert np.allclose(s_dlo, np.outer(correct_dlo, correct_dlo.conj()))



@pytest.mark.filterwarnings("ignore::PendingDeprecationWarning")
def test_basisorder() -> None:
    for b in backends:
        c = Circuit(2)
        c.X(1)
        b.process_circuit(c)
        res = b.run_circuit(c)
        if b.supports_state:
            assert (res.get_state() == np.asarray([0, 1, 0, 0])).all()
            assert (
                res.get_state(basis=BasisOrder.dlo) == np.asarray([0, 0, 1, 0])
            ).all()
        if b.supports_density_matrix:
            sv = np.asarray([0, 1, 0, 0])
            assert (res.get_density_matrix() == np.outer(sv, sv.conj())).all()
            sv = np.asarray([0, 0, 1, 0])
            assert (
                res.get_density_matrix(basis=BasisOrder.dlo) == np.outer(sv, sv.conj())
            ).all()
        c.measure_all()
        res = b.run_circuit(c, n_shots=4, seed=4)
        assert res.get_shots().shape == (4, 2)
        assert res.get_counts() == {(0, 1): 4}


pauli_sym = {"I": Pauli.I, "X": Pauli.X, "Y": Pauli.Y, "Z": Pauli.Z}


def test_measurement_mask() -> None:
    for b in backends:
        n_shots = 10
        circ1 = Circuit(2, 2).X(0).X(1).measure_all()
        circ2 = Circuit(2, 2).X(0).measure_all()
        circ3 = Circuit(2, 1).X(1).Measure(0, 0)
        circ4 = Circuit(3, 2).X(0).Measure(0, 0).Measure(2, 1)
        circ_list = [circ1, circ2, circ3, circ4]
        target_shots = [[1, 1], [1, 0], [0], [1, 0]]

        for i, circ in enumerate(circ_list):
            shots = b.run_circuit(circ, n_shots=n_shots).get_shots()
            for sh in shots:
                assert len(sh) == len(target_shots[i])
                assert np.array_equal(sh, target_shots[i])


#OK
def test_default_pass() -> None:
    for b in backends:
        for ol in range(3):
            comp_pass = b.default_compilation_pass(ol)
            c = Circuit(3, 3)
            c.H(0)
            c.CX(0, 1)
            c.CSWAP(1, 0, 2)
            c.ZZPhase(0.84, 2, 0)
            c.measure_all()
            comp_pass.apply(c)
            for pred in b.required_predicates:
                assert pred.verify(c)


def test_backend_with_circuit_permutation() -> None:
    for b in backends:
        c = Circuit(3).X(0).SWAP(0, 1).SWAP(0, 2)
        qubits = c.qubits
        if b.supports_state:
            sv = b.run_circuit(c).get_state()
        else:
            sv = b.run_circuit(c).get_density_matrix()
        # convert swaps to implicit permutation
        c.replace_SWAPs()
        assert c.implicit_qubit_permutation() == {
            qubits[0]: qubits[1],
            qubits[1]: qubits[2],
            qubits[2]: qubits[0],
        }
        if b.supports_state:
            sv1 = b.run_circuit(c).get_state()
        else:
            sv1 = b.run_circuit(c).get_density_matrix()
        assert np.allclose(sv, sv1, atol=1e-10)
        # test circuits with implicit swaps
        wire_map = {0: 3, 1: 0, 2: 1, 3: 2}
        for x in [0, 1, 2, 3]:
            c = Circuit(4).X(x).SWAP(0, 1).SWAP(1, 2).SWAP(2, 3).measure_all()
            expected_readout = tuple([1 if i == wire_map[x] else 0 for i in range(4)])
            # without implicit swaps
            counts = b.run_circuit(c, n_shots=100).get_counts()
            assert len(counts) == 1
            assert counts[expected_readout] == 100
            # with implicit swaps
            c.replace_SWAPs()
            counts = b.run_circuit(c, n_shots=100).get_counts()
            assert len(counts) == 1
            assert counts[expected_readout] == 100
        # https://github.com/CQCL/pytket-qulacs/issues/86
        c = Circuit(2, 1).X(0).SWAP(0, 1).Measure(1, 0)
        compiled = b.get_compiled_circuit(c, optimisation_level=2)
        res = b.run_circuit(compiled, n_shots=5)
        assert res.get_counts()[(1,)] == 5

#OK
def test_backend_info() -> None:
    for b in backends:
        assert b.backend_info is not None